## Ames housing prices

This project will explore Ames housing dataset and use linear regression to analysis the prices.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
ames = pd.read_csv("AmesHousing.tsv", sep='\t')

In [3]:
# check columns that have more than 25% rows missing
drop_cols = ames.columns[ames.isnull().sum()>round(ames.shape[0]*.25)]
ames[drop_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 5 columns):
Alley           198 non-null object
Fireplace Qu    1508 non-null object
Pool QC         13 non-null object
Fence           572 non-null object
Misc Feature    106 non-null object
dtypes: object(5)
memory usage: 114.5+ KB


In [4]:
def impute_features(df):
    drop_cols = df.columns[df.isnull().sum()>round(df.shape[0]*.25)]
    df = df.drop(drop_cols, 1)
    df = df.select_dtypes(include=['float','int'])
    df = df.fillna(df.mean())
    return df

def transform_features(df):
    # int to categ, scale, fillna, new features (years_until_remod)
    # drop data leaking columns
    d
    return df

def select_features(df):
    
    return df[['Gr Liv Area', 'SalePrice']]
# split data into train test
def train_and_test(df):
    threshold = round(df.shape[0]/2)
    train = df[:threshold]
    test = df[threshold:]
    return train, test

In [32]:
ames1 = impute_features(ames)
train, test = train_and_test(ames1)
# get target and independent variables
target = 'SalePrice'
float_cols = train.select_dtypes(include=['float']).columns
# fit the model
lm = LinearRegression()
lm.fit(train[float_cols], train[target])

test_pred = lm.predict(test[float_cols])
train_pred = lm.predict(train[float_cols])
# get accuracy
test_rmse = np.sqrt(mean_squared_error(test_pred, test[target]))
train_rmse = np.sqrt(mean_squared_error(train_pred, train[target]))

In [33]:
print('Test RMSE:',round(test_rmse, 2), '\nTrain RMSE: ', round(train_rmse,2))

Test RMSE: 53629.22 
Train RMSE:  41888.51


In [151]:
def transform_features(ames):
    # drop columns with more than 25% missing
    df = ames[ames.columns[(ames.isnull().sum()<round(ames.shape[0]*.25))]]

    # drop text columns with any missing rows
    missing_objcol=df.select_dtypes(include=['object']).columns[(df.select_dtypes(include=['object']).isnull().sum()>0)]
    df=df.drop(missing_objcol,1)
    df=df.drop(['PID', 'Order'], 1)
    # create features
    df['years_since_remod'] = df['Yr Sold']-df['Year Remod/Add']
    df['years_since_built'] = df['Yr Sold']-df['Year Built']
    df.drop(['Yr Sold', 'Year Built', 'Year Remod/Add'], axis=1, inplace=True)
    
    # remove low corr fields
    corrdf=(df.corr()['SalePrice'].abs().sort_values())
    corrdf=corrdf[corrdf<0.4]
    df.drop(corrdf.index, 1, inplace=True)
    # maybe drop categories with low variance?
    df = df.fillna(df.mean())
    # get dummies for text columns
    for col in df.select_dtypes(include=['object']):
        if len(df[col].unique())<11:
            col_dummies = pd.get_dummies(df[col])
            df = pd.concat([df, col_dummies], axis='columns')
        del df[col]
    return df


In [153]:
# train a model again
ames1 = transform_features(ames)
train, test = train_and_test(ames1)
# get target and independent variables
target = 'SalePrice'
float_cols = train.drop('SalePrice',1).columns
# fit the model
lm = LinearRegression()
lm.fit(train[float_cols], train[target])

test_pred = lm.predict(test[float_cols])
train_pred = lm.predict(train[float_cols])
# get accuracy
test_rmse = np.sqrt(mean_squared_error(test_pred, test[target]))
train_rmse = np.sqrt(mean_squared_error(train_pred, train[target]))

In [154]:
print('Test RMSE:',round(test_rmse, 2), '\nTrain RMSE: ', round(train_rmse,2))

Test RMSE: 38592.51 
Train RMSE:  22464.16


In [167]:
# lets perform k fold cross-validation:
from sklearn.model_selection import KFold
kf = KFold(5, shuffle=True, random_state=42)

rmse_values=[]
for train_data, test_data in kf.split(ames1):
    train = ames1.iloc[train_data]
    test = ames1.iloc[test_data]
    
    target = 'SalePrice'
    float_cols = train.drop('SalePrice',1).columns
    lm = LinearRegression()
    lm.fit(train[float_cols], train[target])

    test_pred = lm.predict(test[float_cols])
    # get accuracy
    test_rmse = np.sqrt(mean_squared_error(test_pred, test[target]))
    rmse_values.append(test_rmse)
    #print(x,y)
print(rmse_values)
np.mean(rmse_values)

[32114.568990538315, 35859.84530457948, 28149.500147378767, 35508.20762598792, 25487.848990773324]


31423.99421185156

Overall, Our model is giving us RMSE score of 31423 with k=5 cross-validation.